<a href="https://colab.research.google.com/github/VygovskayaNatalya/Lesson1_Python/blob/main/%D0%92%D0%B5%D0%B1%D0%B8%D0%BD%D0%B0%D1%80_25_%D0%B8%D1%8E%D0%BB%D1%8F_%D0%9D%D0%BE%D0%B2%D1%8B%D0%B5_%D1%84%D0%B8%D1%88%D0%BA%D0%B8_chatGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Установка зависимостей

In [ ]:
!pip install openai langchain chromadb faiss-cpu tiktoken pydantic==1.10.8

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 791.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.3/399.3 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 52.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 339.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 83.5 MB

## Авторизация в OpenAI

In [ ]:
#@title Введите OpenAI токен
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

OpenAI API Key:··········


## 1. Динамический выбор из нескольких приглашений (промтов)

Создание цепочки, которая динамически выбирает подсказку для использования при заданном вводе. В частности, мы покажем, как использовать MultiPromptChain для создания цепочки вопросов-ответов, которая выбирает приглашение, наиболее релевантное для данного вопроса, а затем отвечает на вопрос, используя это приглашение.
https://python.langchain.com/docs/modules/chains/additional/multi_prompt_router

In [ ]:
# @title Сервисные функции
from langchain.chains.router import MultiPromptChain
from langchain.chat_models import ChatOpenAI

# Отключение трасировки предупреждающих сообщений при выполнении кода
os.environ["LANGCHAIN_TRACING"] = "false"

In [ ]:
# Подготавливаем темплейты из которых будет осуществлен выбор по описанию для наших задач

physics_template = """Вы очень умный профессор физики. \
У вас отлично получается отвечать на вопросы по физике в краткой и понятной форме. \
Когда вы не знаете ответа на вопрос, вы признаете, что не знаете.

Вот такой вопрос:
{input}"""


math_template = """Вы очень хороший математик. Ты отлично отвечаешь на математические вопросы. \
Вы так хороши, потому что умеете разбивать сложные проблемы на составные части,
отвечать на составные части, а затем сводить их воедино, чтобы ответить на более широкий вопрос.

Вот такой вопрос:
{input}"""

In [ ]:
# Добавляем информацию и описание для промтов

prompt_infos = [
    {
        "name": "physics",
        "description": "Хорошо подходит для ответов на вопросы по физике",
        "prompt_template": physics_template
    },
    {
        "name": "math",
        "description": "Хорошо подходит для ответов на математические вопросы",
        "prompt_template": math_template
    }
]

In [ ]:
chain = MultiPromptChain.from_prompts(ChatOpenAI(), prompt_infos, verbose=True)

In [ ]:
# Запускаем на вопросе из раздела физики (в промежуточных итогах видим что выбран промт physics)

print(chain.run("Что такое излучение черного тела?"))



> Entering new MultiPromptChain chain...
physics: {'input': 'Что такое излучение черного тела?'}
> Finished chain.
Излучение черного тела - это электромагнитное излучение, которое испускает абсолютно черное тело, то есть тело, которое поглощает всю падающую на него энергию без отражения или преломления. Излучение черного тела обладает спектром, который зависит только от его температуры и называется "черным телесным излучением". Это явление было впервые описано Максом Планком в конце 19 века и стало одной из основ физики квантовых явлений.


In [ ]:
# Запускаем на вопросе из раздела математики (в промежуточных итогах видим что выбран промт math)

print(chain.run("Каково первое простое число, большее 40, такое, что единица плюс простое число делится на 3"))



> Entering new MultiPromptChain chain...
math: {'input': 'Какое первое простое число, большее 40, такое, что сумма единицы и этого числа делится на 3?'}
> Finished chain.
Чтобы решить эту задачу, мы можем перебрать все числа, начиная с 41 и проверять, делится ли сумма единицы и числа на 3.

41 + 1 = 42, 42 не делится на 3.
43 + 1 = 44, 44 не делится на 3.
...
47 + 1 = 48, 48 делится на 3.

Таким образом, первое простое число, большее 40, такое, что сумма единицы и этого числа делится на 3, это 47.


## 2. Динамический выбор из нескольких баз знаний - "ретриверов" (векторных хранилищ)
Создание цепочки, которая динамически выбирает, какую поисковую систему использовать. В частности, мы покажем, как использовать MultiRetrievalQAChain для создания цепочки вопросов-ответов, которая выбирает цепочку поиска QA (вопрос-ответ), наиболее подходящую для данного вопроса, а затем отвечает на вопрос, используя ее.
https://python.langchain.com/docs/modules/chains/additional/multi_retrieval_qa_router

In [ ]:
# @title Сервисные функции
from langchain.chains.router import MultiRetrievalQAChain
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import TextLoader
from langchain.vectorstores import FAISS

# Отключение трасировки предупреждающих сообщений при выполнении кода
os.environ["LANGCHAIN_TRACING"] = "false"

!wget -O UII_history_120623.txt "https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/fEwN40_ieKi_0g"
!wget -O UII_learning_120623.txt "https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/v4sae3Lad4Yn-Q"

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
history_docs = TextLoader('/content/UII_history_120623.txt').load()
history_documents = text_splitter.split_documents(history_docs)
history_retriever = FAISS.from_documents(history_documents, OpenAIEmbeddings()).as_retriever()

learning_docs = TextLoader('/content/UII_learning_120623.txt').load()
learning_documents = text_splitter.split_documents(learning_docs)
learning_retriever = FAISS.from_documents(learning_documents, OpenAIEmbeddings()).as_retriever()

personal_texts = [
    "Я люблю яблочный пирог (шарлотку)",
    "Мой любимый цвет фуксии",
    "Моя мечта стать проффесииональным танцором",
    "Я сломал руку, когда мне было 12",
    "Мои родители из России",
]
personal_retriever = FAISS.from_texts(personal_texts, OpenAIEmbeddings()).as_retriever()

--2023-07-21 07:57:33--  https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/fEwN40_ieKi_0g
Resolving getfile.dokpub.com (getfile.dokpub.com)... 142.132.255.217
Connecting to getfile.dokpub.com (getfile.dokpub.com)|142.132.255.217|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://downloader.disk.yandex.ru/disk/e2135a0a95e55cf88e46bd6448364463dcdb708cea35830afd8479cc12cf12dc/64ba6f96/YXKs1LhHLMvs3kY2RY3DzVOd8_9KH5qt4SIPOU-iMb--EvumdB2lgJAS_cA89coCMS_fh2SWcFgc-ch_PJspWw%3D%3D?uid=0&filename=%D0%91%D0%B0%D0%B7%D0%B0%20%D0%B7%D0%BD%D0%B0%D0%BD%D0%B8%D0%B9%20%D0%98%D1%81%D1%82%D0%BE%D1%80%D0%B8%D1%8F%20%D0%A3%D0%98%D0%98.%20%D0%92%D0%B5%D1%80%D1%81%D0%B8%D1%8F%20%D0%BE%D1%82%2012.06.23.txt&disposition=attachment&hash=HfYc21gZK49KTl8aP3w%2BXmjNuCRCGb%2BWWOxn0Mf8s2cvq6un4I9JPPLRSlC4oon2q/J6bpmRyOJonT3VoXnDag%3D%3D%3A&limit=0&content_type=text%2Fplain&owner_uid=813875296&fsize=24339&hid=e0062e361f49634846cd15377c42efd2&media_type=document&tknv=v2

In [ ]:
# Добавляем информацию и описание для наших векторных хранилищ (баз знаний)

retriever_infos = [
    {
        "name": "UII history",
        "description": "Подходит для ответов на вопросы о истории УИИ",
        "retriever": history_retriever # Это база знаний из файла: 'UII_history_120623.txt'
    },
    {
        "name": "UII learning",
        "description": "Подходит для ответов на вопросы об обучении в УИИ",
        "retriever": learning_retriever # Это база знаний из файла: 'UII_learning_120623.txt'
    },
    {
        "name": "personal",
        "description": "Походит для ответов обо мне и моем прошлом",
        "retriever": personal_retriever # Это база знаний из тектса: "Я люблю яблочный пирог (шарлотку)", "Мой любимый цвет фуксии", "Моя мечта стать проффесииональным танцором", "Я сломал руку, когда мне было 12",
    "Мои родители из России"
    }
]


In [ ]:
# Иициализируем класс

chain = MultiRetrievalQAChain.from_retrievers(ChatOpenAI(temperature=0), retriever_infos, verbose=True)

In [ ]:
# Запускаем на вопросе из раздела истории УИИ (в промежуточных итогах видим что выбрано база знаний UII history)

print(chain.run("Когда была получена первая заявка?"))



> Entering new MultiRetrievalQAChain chain...


/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


UII history: {'query': 'Когда была получена первая заявка в УИИ?'}
> Finished chain.
Первая заявка была получена 19 августа 2018 года.


In [ ]:
# Запускаем на вопросе из раздела истории УИИ (в промежуточных итогах видим что выбрано база знаний UII learning)

print(chain.run("Когда сдаются экзамены?"))



> Entering new MultiRetrievalQAChain chain...
UII learning: {'query': 'Когда проходят экзамены в УИИ?'}
> Finished chain.
Экзамены в УИИ проходят один раз в конце обучения.


In [ ]:
# Запускаем на вопросе из раздела истории УИИ (в промежуточных итогах видим что выбрано база знаний personal)

print(chain.run("Что было в моем прошлом?"))



> Entering new MultiRetrievalQAChain chain...
personal: {'query': 'Что было в моем прошлом?'}
> Finished chain.
В вашем прошлом были ваши родители из России, ваша мечта стать профессиональным танцором, вы сломали руку, когда вам было 12 лет, и ваш любимый цвет - фуксия.


## 3. Агенты (Agents)
Агент имеет доступ к набору инструментов и определяет, какие из них использовать, в зависимости от пользовательского ввода.

https://python.langchain.com/docs/modules/agents/

In [ ]:
# @title Сервисные функции
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0)

**Open Meteo API** - Полезно, когда вы хотите получить информацию о погоде из Open Meteo API. Входными данными должен быть вопрос на естественном языке, на который может ответить этот API.

**llm-math или Калькулятор** - Полезно, когда вам нужно ответить на вопросы по математике (используется библиотека math)

In [ ]:
# Определяем наши инструменты
tools = load_tools(["open-meteo-api", "llm-math"], llm=llm)

ZERO_SHOT_REACT_DESCRIPTION - Этот агент использует ReAct фреймворк для определения, какой инструмент использовать, исключительно на основе описания инструмента. Может быть предоставлено любое количество инструментов. Этот агент требует, чтобы для каждого инструмента было предоставлено описание.

**Примечание: Это агент действия наиболее общего назначения.**

Полный список типов агентов см. в разделе https://python.langchain.com/docs/modules/agents/agent_types/

verbose - это параметр отоборажения или скрытия промежуточной информации

In [ ]:
# Инициализируем агента
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

Можно справшивать по отдельности, и в зависимости от вопросов будет использоваться тот или иной инструмент (у нас в данном случае есть инструмент API метео сервиса для информации о погоде, и математический основанный на библиотеке math)

In [ ]:
# Запускаем на вопросе про погоду (в промежуточных итогах видим что выбрано действие Action: Open Meteo API)

agent.run("Какая погода в Сочи и скорость ветра? Ответь на руссском языке")



> Entering new  chain...
I need to use the Open Meteo API to get the weather information for Sochi.
Action: Open Meteo API
Action Input: "Какая погода в Сочи?"
Observation: The current weather in Sochi is 22°C with a windspeed of 20.1 km/h coming from the southeast (151°). The weather condition is represented by the WMO code 95, indicating a specific weather condition. It is currently daytime in Sochi. The time of the weather report is 07:00 on July 10, 2023.
Thought:I now know the weather in Sochi and the wind speed.
Final Answer: В Сочи сейчас 22°C, скорость ветра составляет 20.1 км/ч.

> Finished chain.


'В Сочи сейчас 22°C, скорость ветра составляет 20.1 км/ч.'

In [ ]:
# Запускаем на задачке по математике (в промежуточных итогах видим что выбрано действие Action: Calculator)

agent.run("Пол комнаты, имеющей форму прямоугольника со сторонами 3 м и строной 9 м, требуется покрыть паркетом из прямоугольных дощечек со сторонами 5 см и 30 см. Сколько потребуется таких дощечек? Ответь на руссском языке")



> Entering new  chain...
Я должен рассчитать площадь комнаты и площадь одной дощечки, чтобы определить количество необходимых дощечек.
Action: Calculator
Action Input: Рассчитать площадь комнаты: 3 м * 9 м
Observation: Answer: 27
Thought:Теперь я знаю площадь комнаты - 27 квадратных метров. Теперь мне нужно рассчитать площадь одной дощечки.
Action: Calculator
Action Input: Рассчитать площадь одной дощечки: 5 см * 30 см
Observation: Answer: 150
Thought:Теперь я знаю площадь комнаты - 27 квадратных метров и площадь одной дощечки - 150 квадратных сантиметров. Теперь мне нужно рассчитать количество необходимых дощечек.
Action: Calculator
Action Input: Рассчитать количество дощечек: 27 квадратных метров / 150 квадратных сантиметров
Observation: Answer: 1800.0
Thought:Теперь я знаю, что для покрытия пола комнаты потребуется 1800 дощечек.
Final Answer: Для покрытия пола комнаты потребуется 1800 дощечек.

> Finished chain.


'Для покрытия пола комнаты потребуется 1800 дощечек.'

Можно в одном запросе перечислить несколько вопросов из одной или разных тем, соответственно применится тот или иной инструмент.

In [ ]:
agent.run("Сколько я потрачу бензина, при расходе моего автомобиля 7 л на 100 км в загородном режиме и 9.5 л на 100 км в городском режиме чтобы проехать 874 км по трассе и 125 км по городу? Какая погода в Сочи?")



> Entering new AgentExecutor chain...
I need to calculate the amount of gasoline used for both highway and city driving, and then find out the weather in Sochi.
Action: Calculator
Action Input: (874 km / 100 km) * 7 L + (125 km / 100 km) * 9.5 L
Observation: Answer: 73.055
Thought:Now I need to find out the weather in Sochi.
Action: Open Meteo API
Action Input: "What is the weather in Sochi?"
Observation: The current weather in Sochi is 26.4°C with a windspeed of 17.6 km/h coming from the northwest. The weather condition is categorized as code 3, indicating a specific weather condition. The time of the weather report is 15:00 on July 24, 2023.
Thought:I now know the final answer.
Final Answer: The amount of gasoline needed to travel 874 km on the highway and 125 km in the city is 73.055 liters. The current weather in Sochi is 26.4°C with a windspeed of 17.6 km/h coming from the northwest. The weather condition is code 3, and the report was made at 15:00 on July 24, 2023.

> Finished 

'The amount of gasoline needed to travel 874 km on the highway and 125 km in the city is 73.055 liters. The current weather in Sochi is 26.4°C with a windspeed of 17.6 km/h coming from the northwest. The weather condition is code 3, and the report was made at 15:00 on July 24, 2023.'

In [ ]:
agent.run("Какая сейчас погода в Сочи? Какая погода сегодня в Пхукете Тайланд?")



> Entering new  chain...
I need to use the Open Meteo API to get the weather information for Sochi and Phuket.
Action: Open Meteo API
Action Input: "Какая сейчас погода в Сочи?"
Observation: The current weather in Sochi is 21.8°C with a windspeed of 18.4 km/h coming from the southeast direction. The weather condition is represented by the WMO code 95, indicating a specific weather condition. The observation was made at 08:00 local time on July 10, 2023.
Thought:I have the weather information for Sochi. Now I need to get the weather information for Phuket.
Action: Open Meteo API
Action Input: "Какая погода сегодня в Пхукете Тайланд?"
Observation: The current weather in Phuket, Thailand is 28.2°C with a windspeed of 11.5 km/h coming from the west (266°). The weather condition is cloudy (weathercode 80) and it is currently daytime. The time of the weather report is 08:00 on July 10, 2023.
Thought:I now have the weather information for both Sochi and Phuket.
Final Answer: The current wea

'The current weather in Sochi is 21.8°C with a windspeed of 18.4 km/h, and the current weather in Phuket is 28.2°C with a windspeed of 11.5 km/h.'

## 4. HTTP Chain
Использование библиотеки request для получения результатов HTML из URL, а затем передачу в LLM для анализа результатов.

https://python.langchain.com/docs/modules/chains/additional/llm_requests

In [ ]:
# @title Сервисные функции
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMRequestsChain, LLMChain
from langchain.prompts import PromptTemplate

def get_input(question: str):
  inputs = {
    "query": question,
    "url": "https://www.google.com/search?q=" + question.replace(" ", "+"),
    }
  return inputs

Поиск в Google (на Yandex не работает, надо изменить template под используемый парсер)

In [ ]:
# Подготавливаем темплейт для формирования запроса и получения ответа

template = """Between >>> and <<< are the raw search result text from google.
Extract the answer to the question '{query}' or say "not found" if the information is not contained.
Use the format
Extracted:<answer or "not found">
>>> {requests_result} <<<
Extracted:"""

PROMPT = PromptTemplate(
    input_variables=["query", "requests_result"],
    template=template,
)

In [ ]:
# Инициализируем класс

chain = LLMRequestsChain(llm_chain=LLMChain(llm=ChatOpenAI(temperature=0), prompt=PROMPT))

In [ ]:
# Запускаем на вопросе

question = get_input("Напиши три (3) крупнейшие страны и их размеры в км2?")

result = chain(question)
print(result)

{'query': 'Напиши три (3) крупнейшие страны и их размеры в км2?', 'url': 'https://www.google.com/search?q=Напиши+три+(3)+крупнейшие+страны+и+их+размеры+в+км2?', 'output': 'Россия (РФ) - 17 125 191 км²\nКанада - 9 984 670 км²\nКитай (КНР) - 9 598 962 км²'}


In [ ]:
# Вывод только ответа

print(result['output'])

Россия (РФ) - 17 125 191 км²
Канада - 9 984 670 км²
Китай (КНР) - 9 598 962 км²


In [ ]:
question = get_input("Основатель Университета исскуственного интеллекта?")

result = chain(question)
print(result)

{'query': 'Основатель Университета исскуственного интеллекта?', 'url': 'https://www.google.com/search?q=Основатель+Университета+исскуственного+интеллекта?', 'output': 'Extracted: Дмитрий Романов'}


In [ ]:
print(result['output'])

Extracted: Дмитрий Романов


## 5. SQL
Пример демонстрирует использование SQLDatabaseChain для ответов на вопросы через базу данных SQL и ее изменения (удаление, редактирование, добавление записей) с помощью запросов на естественном языке.

https://python.langchain.com/docs/modules/chains/popular/sqlite

Загрузка демонстрационной базы данных Chinook.

База данных Chinook была создана как альтернатива базе данных Northwind. Она представляет собой хранилище цифровых мультимедиа, включающее таблицы для исполнителей, альбомов, мультимедийных дорожек, счетов-фактур и клиентов.

In [ ]:
# @title Сервисные функции
from langchain import OpenAI, SQLDatabase, SQLDatabaseChain
from langchain.chat_models import ChatOpenAI

# Отключение трасировки предупреждающих сообщений при выполнении кода
os.environ["LANGCHAIN_TRACING"] = "false"

!wget https://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip
!unzip chinook.zip

db = SQLDatabase.from_uri("sqlite://///content/chinook.db")
llm = ChatOpenAI(temperature=0, verbose=True)

--2023-07-10 10:34:24--  https://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip
Resolving www.sqlitetutorial.net (www.sqlitetutorial.net)... 151.139.128.10
Connecting to www.sqlitetutorial.net (www.sqlitetutorial.net)|151.139.128.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 305596 (298K) [application/zip]
Saving to: ‘chinook.zip’

chinook.zip         100%[===================>] 298.43K  --.-KB/s    in 0.04s   

2023-07-10 10:34:24 (7.10 MB/s) - ‘chinook.zip’ saved [305596/305596]



In [ ]:
# Инициализируем класс

db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [ ]:
# Вывод информации о таблицах базы данных Chinook

print(db.table_info)


CREATE TABLE albums (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES artists ("ArtistId")
)

/*
3 rows from albums table:
AlbumId	Title	ArtistId
1	For Those About To Rock We Salute You	1
2	Balls to the Wall	2
3	Restless and Wild	2
*/


CREATE TABLE artists (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from artists table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/


CREATE TABLE customers (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60) NOT NULL, 
	"SupportRepId" INTEGER, 
	PRIMARY KEY ("CustomerId"), 
	FOREIGN KEY("SupportRepId") REFERENCES employe

In [ ]:
# Запускаем на запросе удаления удаления данных из таблицы (из словесного описания сформирован sql запрос (SQLQuery:) на удаление)

db_chain.run("Удали первую запись из таблицы работников?")



> Entering new  chain...
Удали первую запись из таблицы работников?
SQLQuery:DELETE FROM employees WHERE EmployeeId = (SELECT MIN(EmployeeId) FROM employees)
SQLResult: 
Answer:The first record from the employees table has been deleted.
> Finished chain.


'The first record from the employees table has been deleted.'

In [ ]:
# Запускаем на запросе о получении информации из таблицы (из словесного описания сформирован sql запрос (SQLQuery:) на выборку информации)

db_chain.run("Назовите несколько примеров композиций Bach?")



> Entering new  chain...
Назовите несколько примеров композиций Bach?
SQLQuery:SELECT "Name" 
FROM tracks 
WHERE "Composer" LIKE '%Bach%'
LIMIT 5;
SQLResult: [('American Woman',), ('Concerto for 2 Violins in D Minor, BWV 1043: I. Vivace',), ('Aria Mit 30 Veränderungen, BWV 988 "Goldberg Variations": Aria',), ('Suite for Solo Cello No. 1 in G Major, BWV 1007: I. Prélude',), ('Toccata and Fugue in D Minor, BWV 565: I. Toccata',)]
Answer:('American Woman',), ('Concerto for 2 Violins in D Minor, BWV 1043: I. Vivace',), ('Aria Mit 30 Veränderungen, BWV 988 "Goldberg Variations": Aria',), ('Suite for Solo Cello No. 1 in G Major, BWV 1007: I. Prélude',), ('Toccata and Fugue in D Minor, BWV 565: I. Toccata',)
> Finished chain.


'(\'American Woman\',), (\'Concerto for 2 Violins in D Minor, BWV 1043: I. Vivace\',), (\'Aria Mit 30 Veränderungen, BWV 988 "Goldberg Variations": Aria\',), (\'Suite for Solo Cello No. 1 in G Major, BWV 1007: I. Prélude\',), (\'Toccata and Fugue in D Minor, BWV 565: I. Toccata\',)'

In [ ]:
# Запускаем на запросе о получении информации из таблицы (из словесного описания сформирован sql запрос (SQLQuery:) на выборку информации)

db_chain.run("Сколько альбомов у Aerosmith?")



> Entering new  chain...
Сколько альбомов у Aerosmith?
SQLQuery:SELECT COUNT(*) AS "Number of Albums"
FROM albums
JOIN artists ON albums.ArtistId = artists.ArtistId
WHERE artists.Name = 'Aerosmith'
SQLResult: [(1,)]
Answer:Сколько альбомов у Aerosmith?
Answer: У Aerosmith 1 альбом.
> Finished chain.


'Сколько альбомов у Aerosmith?\nAnswer: У Aerosmith 1 альбом.'

In [ ]:
# Запускаем на запросе о получении информации из таблицы (из словесного описания сформирован sql запрос (SQLQuery:) на выборку информации)

db_chain.run("Сколько сотрудников приживает в Канаде?")



> Entering new  chain...
Сколько сотрудников приживает в Канаде?
SQLQuery:SELECT COUNT(*) FROM employees WHERE Country = 'Canada'
SQLResult: [(8,)]
Answer:8
> Finished chain.


'8'

In [ ]:
# Вариант вывода результата без промежуточной информации

db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=False)
db_chain.run("Сколько сотрудников приживает в Канаде?")

'8'

## 6. Поиск информации для ответа в разговоре QA и документу

In [ ]:
# @title Сервисные функции
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma, FAISS
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.memory import ConversationBufferMemory

# Загрузка документа История УИИ
!wget -O UII_history_120623.txt "https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/fEwN40_ieKi_0g"

# Создание векторной базы из документа История УИИ
loader = TextLoader("/content/UII_history_120623.txt")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(documents, embeddings)

# Инициализация памяти истории чата
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

def get_input(question: str):
  inputs = {"question": query}
  return inputs

--2023-07-11 08:29:47--  https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/fEwN40_ieKi_0g
Resolving getfile.dokpub.com (getfile.dokpub.com)... 142.132.255.217
Connecting to getfile.dokpub.com (getfile.dokpub.com)|142.132.255.217|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://downloader.disk.yandex.ru/disk/674c5c18f5125c10fd88ec627cfcb5e013a50027bd6922f08a9f55ae80004518/64ad4b3c/YXKs1LhHLMvs3kY2RY3DzVOd8_9KH5qt4SIPOU-iMb--EvumdB2lgJAS_cA89coCMS_fh2SWcFgc-ch_PJspWw%3D%3D?uid=0&filename=%D0%91%D0%B0%D0%B7%D0%B0%20%D0%B7%D0%BD%D0%B0%D0%BD%D0%B8%D0%B9%20%D0%98%D1%81%D1%82%D0%BE%D1%80%D0%B8%D1%8F%20%D0%A3%D0%98%D0%98.%20%D0%92%D0%B5%D1%80%D1%81%D0%B8%D1%8F%20%D0%BE%D1%82%2012.06.23.txt&disposition=attachment&hash=HfYc21gZK49KTl8aP3w%2BXmjNuCRCGb%2BWWOxn0Mf8s2cvq6un4I9JPPLRSlC4oon2q/J6bpmRyOJonT3VoXnDag%3D%3D%3A&limit=0&content_type=text%2Fplain&owner_uid=813875296&fsize=24339&hid=e0062e361f49634846cd15377c42efd2&media_type=document&tknv=v2

### 1) Цепочка ConversationalRetrievalQA основана на цепочке RetrievalQAChain для обеспечения компонента истории чата.

Сначала он объединяет историю чата (либо явно переданную, либо извлеченную из предоставленной памяти) и вопрос в отдельный вопрос, затем просматривает соответствующие документы у ретривера и, наконец, передает эти документы и вопрос в цепочку ответов на вопросы, чтобы вернуть ответ.

In [ ]:
# Инициализируем класс

qa = ConversationalRetrievalChain.from_llm(ChatOpenAI(temperature=0), vectorstore.as_retriever(), memory=memory)

In [ ]:
# Запускаем на вопросе

question = get_input("Привет! Меня зовут Аркадий Аркадьевич, в каком году запущена певая версия Terra AI?")
result = qa(question)

#Вывод результата ответа модели

print(result["answer"])

 Первая версия Terra AI была запущена в октябре 2021 года.


In [ ]:
# Просмотр результата включая промежуточные данные и историю сообщений

result

{'question': 'Привет! Меня зовут Аркадий Аркадьевич, в каком году запущена певая версия Terra AI?',
 'chat_history': [HumanMessage(content='Привет! Меня зовут Аркадий Аркадьевич, в каком году запущена певая версия Terra AI?', additional_kwargs={}, example=False),
  AIMessage(content=' Первая версия Terra AI была запущена в октябре 2021 года.', additional_kwargs={}, example=False)],
 'answer': ' Первая версия Terra AI была запущена в октябре 2021 года.'}

In [ ]:
question = get_input("Какие книги нравятся Дмитрию Романову?")
result = qa(question)

print(result["answer"])

 Дмитрий Романов любит книги по нейронным сетям и психологии.


In [ ]:
# Просмотр результата включая промежуточные данные и историю сообщений

result

{'question': 'Какие книги нравятся Дмитрию Романову?',
 'chat_history': [HumanMessage(content='Привет! Меня зовут Аркадий Аркадьевич, в каком году запущена певая версия Terra AI?', additional_kwargs={}, example=False),
  AIMessage(content=' Первая версия Terra AI была запущена в октябре 2021 года.', additional_kwargs={}, example=False),
  HumanMessage(content='Какие книги нравятся Дмитрию Романову?', additional_kwargs={}, example=False),
  AIMessage(content=' Дмитрий Романов любит книги по нейронным сетям и психологии.', additional_kwargs={}, example=False)],
 'answer': ' Дмитрий Романов любит книги по нейронным сетям и психологии.'}

In [ ]:
question = get_input("Какими языками программмирования он владеет?")
result = qa(question)

print(result["answer"])

 Дмитрий Романов владеет языками программирования Pascal, C/C+, C#, Wpf, Silverlight и Python.


In [ ]:
# Просмотр результата включая промежуточные данные и историю сообщений

result

{'question': 'Какими языками программмирования он владеет?',
 'chat_history': [HumanMessage(content='Привет! Меня зовут Аркадий Аркадьевич, в каком году запущена певая версия Terra AI?', additional_kwargs={}, example=False),
  AIMessage(content=' Первая версия Terra AI была запущена в октябре 2021 года.', additional_kwargs={}, example=False),
  HumanMessage(content='Какие книги нравятся Дмитрию Романову?', additional_kwargs={}, example=False),
  AIMessage(content=' Дмитрий Романов любит книги по нейронным сетям и психологии.', additional_kwargs={}, example=False),
  HumanMessage(content='Какими языками программмирования он владеет?', additional_kwargs={}, example=False),
  AIMessage(content=' Дмитрий Романов владеет языками программирования Pascal, C/C+, C#, Wpf, Silverlight и Python.', additional_kwargs={}, example=False)],
 'answer': ' Дмитрий Романов владеет языками программирования Pascal, C/C+, C#, Wpf, Silverlight и Python.'}

### 2) Использование разных моделей для уточнения вопроса и генерации ответа

Эта цепочка состоит из двух этапов. Во-первых, она объединяет текущий вопрос и историю чата в отдельный вопрос. Это необходимо для создания отдельного вектора для использования при поиске. После этого он выполняет поиск, а затем отвечает на вопрос, используя расширенную генерацию поиска с помощью отдельной модели.

GPT4 для генерации ответа, а ChatGpt (более дешевая)  для уточнения вопроса и отбора информации из истории и документа

In [ ]:
# Инициализируем класс

qa = ConversationalRetrievalChain.from_llm(
    ChatOpenAI(temperature=0, model="gpt-4"),
    vectorstore.as_retriever(),
    condense_question_llm = ChatOpenAI(temperature=0, model='gpt-3.5-turbo'),
)
# Инициализируем историю чата

chat_history = []

In [ ]:
# Запускаем на вопросе

question = "Привет! Меня зовут Аркадий Аркадьевич, в каком году запущена певая версия Terra AI?"
result = qa({"question": question, "chat_history": chat_history})

print(result["answer"])

Первая версия Terra AI была запущена в октябре 2021 года.


In [ ]:
result

{'question': 'Привет! Меня зовут Аркадий Аркадьевич, в каком году запущена певая версия Terra AI?',
 'chat_history': [],
 'answer': 'Первая версия Terra AI была запущена в октябре 2021 года.'}

In [ ]:
chat_history.append((question, result["answer"]))
question = "Какие книги нравятся Дмитрию Романову?"
result = qa({"question": question, "chat_history": chat_history})

print(result["answer"])

Дмитрию Романову нравятся книги по нейронным сетям.


In [ ]:
result

{'question': 'Какие книги нравятся Дмитрию Романову?',
 'chat_history': [('Привет! Меня зовут Аркадий Аркадьевич, в каком году запущена певая версия Terra AI?',
   'Первая версия Terra AI была запущена в октябре 2021 года.')],
 'answer': 'Дмитрию Романову нравятся книги по нейронным сетям.'}

In [ ]:
chat_history.append((question, result["answer"]))
question = "Какими языками программмирования он владеет?"
result = qa({"question": question, "chat_history": chat_history})

print(result["answer"])

Дмитрий Романов владеет языками программирования Pascal, C/C+, C#, а также Matlab, который является языком типа Python.


In [ ]:
result

{'question': 'Какими языками программмирования он владеет?',
 'chat_history': [('Привет! Меня зовут Аркадий Аркадьевич, в каком году запущена певая версия Terra AI?',
   'Первая версия Terra AI была запущена в октябре 2021 года.'),
  ('Какие книги нравятся Дмитрию Романову?',
   'Дмитрию Романову нравятся книги по нейронным сетям.')],
 'answer': 'Дмитрий Романов владеет языками программирования Pascal, C/C+, C#, а также Matlab, который является языком типа Python.'}

In [ ]:
chat_history.append((question, result["answer"]))
question = "Почему он провел первую живую лекцию сам?"
result = qa({"question": question, "chat_history": chat_history})

print(result["answer"])

Дмитрий Романов провел первую живую лекцию сам, потому что один из преподавателей не смог провести это занятие.


In [ ]:
result

{'question': 'Почему он провел первую живую лекцию сам?',
 'chat_history': [('Привет! Меня зовут Аркадий Аркадьевич, в каком году запущена певая версия Terra AI?',
   'Первая версия Terra AI была запущена в октябре 2021 года.'),
  ('Какие книги нравятся Дмитрию Романову?',
   'Дмитрию Романову нравятся книги по нейронным сетям.'),
  ('Какими языками программмирования он владеет?',
   'Дмитрий Романов владеет языками программирования Pascal, C/C+, C#, а также Matlab, который является языком типа Python.')],
 'answer': 'Дмитрий Романов провел первую живую лекцию сам, потому что один из преподавателей не смог провести это занятие.'}

## 7. GPT4 извлечение сущностей
Цепочка извлечения использует параметр OpenAI functions, чтобы указать схему для извлечения объектов из документа. Это помогает нам убедиться, что модель выводит именно ту схему объектов и свойств, которые мы хотим, с их соответствующими типами.

Цепочку извлечения следует использовать, когда мы хотим извлечь несколько объектов с их свойствами из одного и того же отрывка (т. Е. какие люди были упомянуты в этом отрывке?)

In [ ]:
# @title Сервисные функции
from langchain.chat_models import ChatOpenAI
from langchain.chains import create_extraction_chain, create_extraction_chain_pydantic
from langchain.prompts import ChatPromptTemplate

from typing import Optional, List
from pydantic import BaseModel, Field

llm = ChatOpenAI(temperature=0, model="gpt-4")

# Либо с помощью ChatGpt
# llm = ChatOpenAI(temperature=0, model="gpt-3.5")


In [ ]:
# Это наш текст из которого мы будем извлекать именованные сущности

input = """
Рост Владимира - 187 см. Евгения на 7 см выше Владимира и прыгает выше него. Евгения - брюнетка, а Владимир - блондин.
Собака Владимира Конни - лабрадор, и она любит играть в прятки.
        """

### 1) **с помощью схемы**

Чтобы извлечь сущности, нам нужно создать схему, подобную следующей, в которой мы указываем все свойства, которые мы хотим найти, и тип, который мы ожидаем, что они будут иметь. Мы также можем указать, какие из этих свойств являются обязательными, а какие необязательными.

In [ ]:
# Создаем схему структуры наших параметров

schema = {
    "properties": {
        "person_name": {"type": "string"},
        "person_height": {"type": "integer"},
        "person_hair_color": {"type": "string"},
        "dog_name": {"type": "string"},
        "dog_breed": {"type": "string"},
    },
    "required": ["person_name", "person_height"],
}

chain = create_extraction_chain(schema, llm)

In [ ]:
# Запускаем на тексте (получаем структуру найденных сущностей согласно созданной схеме)

chain.run(input)

[{'person_name': 'Владимир',
  'person_height': 187,
  'person_hair_color': 'блондин',
  'dog_name': 'Конни',
  'dog_breed': 'лабрадор'},
 {'person_name': 'Евгения',
  'person_height': 194,
  'person_hair_color': 'брюнетка'}]

### 2) **с помощью pydantic**

Мы также можем использовать схему Pydantic для выбора требуемых свойств и типов, и мы установим как "Необязательные" те, которые не являются строго обязательными.

Используя create_extraction_chain_pydantic функцию, мы можем отправить Pydantic schema в качестве входных данных, а на выходе будет создан экземпляр объекта, который соответствует нашей желаемой схеме.

Таким образом, мы можем указать нашу схему таким же образом, как мы определили бы новый класс или функцию в Python - с чисто питоновскими типами.

In [ ]:
# Создаем класс наших параметров

class Properties(BaseModel):
    person_name: str
    person_height: int
    person_hair_color: str
    dog_breed: Optional[str]
    dog_name: Optional[str]

chain = create_extraction_chain_pydantic(pydantic_schema=Properties, llm=llm)

In [ ]:
# Запускаем на тексте (получаем типизированную структуру)

chain.run(input)

[Properties(person_name='Владимир', person_height=187, person_hair_color='блондин', dog_breed='лабрадор', dog_name='Конни'),
 Properties(person_name='Евгения', person_height=194, person_hair_color='брюнетка', dog_breed='', dog_name='')]

## 8. Чат с GPT4

In [ ]:
# @title Сервисные функции
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain

llm = ChatOpenAI(temperature=0, model="gpt-4")

# Установка verbose в True приведет к распечатке некоторых внутренних состояний Chain объекта во время его запуска.
conversation = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory(),
    verbose=True
)

In [ ]:
# Запуск диалога с нашим вопросом (в промежуточном выводе сохраняется диалог)

conversation.run("Привет я Анатолий. Что такое ChatGpt?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Примет я Анатолий. Что такое ChatGpt?
AI:

> Finished chain.


'Привет, Анатолий! ChatGPT - это модель искусственного интеллекта, разработанная OpenAI. "GPT" означает "Generative Pretrained Transformer", что можно перевести как "Генеративный предварительно обученный трансформер". Это модель, которая использует машинное обучение для понимания и генерации текста. Она была обучена на множестве текстов из интернета и может генерировать ответы на вопросы, писать эссе, создавать истории и многое другое. Однако стоит помнить, что, несмотря на свою способность генерировать информативные и креативные ответы, ChatGPT не обладает собственным сознанием или пониманием. Она просто анализирует введенный текст и генерирует ответ на основе своего обучения.'

In [ ]:
conversation.run("Как ее можно использовать?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Примет я Анатолий. Что такое ChatGpt?
AI: Привет, Анатолий! ChatGPT - это модель искусственного интеллекта, разработанная OpenAI. "GPT" означает "Generative Pretrained Transformer", что можно перевести как "Генеративный предварительно обученный трансформер". Это модель, которая использует машинное обучение для понимания и генерации текста. Она была обучена на множестве текстов из интернета и может генерировать ответы на вопросы, писать эссе, создавать истории и многое другое. Однако стоит помнить, что, несмотря на свою способность генерировать информативные и креативные ответы, ChatGPT не обладает собственным сознанием или пониманием. Она просто анализирует введенный текст и ген

'ChatGPT можно использовать в различных областях и для различных задач. Вот некоторые примеры:\n\n1. Помощник в написании текстов: ChatGPT может помочь в написании статей, блогов, эссе и других текстов, предлагая идеи или продолжая начатый вами текст.\n\n2. Обучение и образование: ChatGPT может использоваться как инструмент для обучения, отвечая на вопросы студентов, помогая в выполнении домашних заданий или объясняя сложные концепции.\n\n3. Поддержка клиентов: ChatGPT может быть интегрирована в системы поддержки клиентов для автоматического ответа на часто задаваемые вопросы.\n\n4. Игры и развлечения: ChatGPT может создавать интерактивные истории или игры, где пользователи могут взаимодействовать с AI.\n\n5. Персональный ассистент: ChatGPT может помочь в организации вашего дня, напоминая о важных событиях или задачах.\n\nОднако, несмотря на все эти возможности, важно помнить, что ChatGPT не обладает собственным сознанием или пониманием, и все ответы генерируются на основе обучения на 

In [ ]:
conversation.run("Напомни как меня зовут?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Примет я Анатолий. Что такое ChatGpt?
AI: Привет, Анатолий! ChatGPT - это модель искусственного интеллекта, разработанная OpenAI. "GPT" означает "Generative Pretrained Transformer", что можно перевести как "Генеративный предварительно обученный трансформер". Это модель, которая использует машинное обучение для понимания и генерации текста. Она была обучена на множестве текстов из интернета и может генерировать ответы на вопросы, писать эссе, создавать истории и многое другое. Однако стоит помнить, что, несмотря на свою способность генерировать информативные и креативные ответы, ChatGPT не обладает собственным сознанием или пониманием. Она просто анализирует введенный текст и ген

'Тебя зовут Анатолий.'

## 9. Вызов функции в OpenAI
В вызове API вы можете описать функции в gpt-3.5-turbo-0613 и gpt-4-0613 и заставить модель разумно выбрать вывод объекта JSON, содержащего аргументы для вызова этих функций. API не вызывает функцию; вместо этого модель генерирует JSON, который вы можете использовать для вызова функции в своем коде.


**Пример использования выделения сущностей из текста и  вызова функций в моделях OpenAi ведения таблицы о ваших тратах (покупках)**

In [ ]:
# @title Сервисные функции
import openai
import json
import pandas as pd

# Пример фиктивной функции, запрограммированной на добавление записи о ваших расходах
# В рабочей среде это может быть ваш внутренний API или внешний API
def add_current_sale(location, count, product_name):
    """Добавить запись в таблицу расходов"""
    df.loc[len(df.index)] = [location, count, product_name]
    sales_info = {
        "Место покупки": location,
        "Количество": count,
        "Наименование": product_name,
        "Статус": "Записано!"
    }
    df.head()
    return json.dumps(sales_info)

def run_conversation(query):
# Шаг 1: отправьте разговор и доступные функции в GPT
    messages = [{"role": "system", "content": "Ты вежливый Ассистент."},
      {"role": "user", "content": query}]

    functions = [
        {
            "name": "add_current_sale",
            "description": "Добавить запись в таблицу расходов.",
            "parameters": {
                "type": "object",
                "properties": {
                    "Место покупки": {
                        "type": "string",
                        "description": "Место где потрачены деньги, например магазин (Пятерочка), пекарня, заправка и т.д.",
                    },
                    "Количество": {"type": "string",
                    "description": "Количество купленного товара, например 1, 5 бутылок, 4 куска, 2 штуки, 1 пакет и т.д.",
                                   },

                    "Наименование": {"type": "string",
                    "description": "Наименование купленного товара, например вода без газа, минералка Есентуки, масло сливочное, бензин и т.д.",
                                     },
                    },
                "required": ["Место покупки", "Количество", "Наименование"],
            },
        },
    ]
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        functions=functions,
        function_call="auto",  # по умолчанию используется значение auto
    )
    response_message = response["choices"][0]["message"]

    # Шаг 2: проверьте, хочет ли GPT вызвать функцию
    if response_message.get("function_call"):
        # Шаг 3: вызовите функцию
        # Примечание: ответ JSON не всегда может быть действительным; обязательно обработайте ошибки
        available_functions = {
            "add_current_sale": add_current_sale,
        }  # в этом примере только одна функция, но у вас может быть несколько
        function_name = response_message["function_call"]["name"]
        fuction_to_call = available_functions[function_name]
        function_args = json.loads(response_message["function_call"]["arguments"])
        print(function_args)
        function_response = fuction_to_call(
            location=function_args.get("Место покупки"),
            count=function_args.get("Количество"),
            product_name=function_args.get("Наименование"),
        )

        # Шаг 4: отправьте информацию о вызове функции и ответе функции в GPT
        messages.append(response_message)  # продлите разговор с помощью ответа помощника
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # расширьте диалог с помощью function_response
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages,
        )  # получите новый ответ от GPT, где вы сможете увидеть ответ функции
        return second_response["choices"][0]["message"]["content"]
    return response_message["content"]

Создадим тестовые данные

In [ ]:
# Создадим тестовую таблицу расходов с некоторыми данными

data = [
    ["Пекарня", '5 штук', "круасаны"],
    ["Пятерочка", '1 бутылка', "вода без газа"]
    ]

df = pd.DataFrame(data, columns=["Место покупки", "Количество", "Наименование"])
df

,Место покупки,Количество,Наименование
0,Пекарня,5 штук,круасаны
1,Пятерочка,1 бутылка,вода без газа


Пример добавления записи о ваших расходах в dataframe Pandas

In [ ]:
#Запуск модели с вашими данными о покупках (в промежуточном выводе отображается структура найденных данных и ответ модели)

print(run_conversation('Я сегодня был в Пятерочке, и купил 7 бутылок кефира. Добавь в таблицу расходов'))

{'Место покупки': 'Пятерочка', 'Количество': '7', 'Наименование': 'кефир'}
Я добавил информацию о покупке в таблицу расходов. Запись была успешно добавлена! Если у вас есть еще какие-либо записи, которые вы хотели бы добавить, пожалуйста, сообщите мне.


In [ ]:
# Просмотр таблицы на наличие новой записи

df

,Место покупки,Количество,Наименование
0,Пекарня,5 штук,круасаны
1,Пятерочка,1 бутылка,вода без газа
2,Пятерочка,7,кефир


Данный запрос не вызывает нашу функцию а просто генерирует ответ

In [ ]:
#Запуск модели с данными не относящимися к покупкам (в промежуточном выводе отображается только ответ модели)

print(run_conversation('Напиши пару строк про искуственный интеллект'))

Искусственный интеллект (ИИ) – это область компьютерной науки, которая занимается созданием и развитием программ и систем, способных выполнять задачи, требующие обычно человеческого интеллекта. ИИ включает в себя различные методы и подходы, такие как машинное обучение, глубокое обучение, обработка естественного языка и робототехника. Успехи в области искусственного интеллекта привели к разработке таких систем, как голосовые помощники, автономные автомобили, системы медицинской диагностики и многое другое. Искусственный интеллект имеет огромный потенциал для изменения нашей жизни и решения сложных проблем в различных областях.


In [ ]:
# Просмотр таблицы на отсутствие новой записи

df

,Место покупки,Количество,Наименование
0,Пекарня,5 штук,круасаны
1,Пятерочка,1 бутылка,вода без газа
2,Пятерочка,7,кефир


In [ ]:
#Запуск модели с вашими данными о покупках (в промежуточном выводе отображается структура найденных данных и ответ модели)

print(run_conversation('Весь день пробыл в дороге, заезжал на заправку, и купил 41 литр бензина.'))

{'Место покупки': 'Заправка', 'Количество': '41 литр', 'Наименование': 'Бензин'}
Запись о покупке 41 литра бензина на заправке успешно добавлена.


In [ ]:
# Просмотр таблицы на наличие новой записи

df

,Место покупки,Количество,Наименование
0,Пекарня,5 штук,круасаны
1,Пятерочка,1 бутылка,вода без газа
2,Пятерочка,7,кефир
3,Заправка,41 литр,Бензин
